# Final Project: Hillary Clinton's Performance by State
The goal of my final project is to find the average number of percentage points Hillary Clinton lost by in each state for the Democratic Primary against Bernie Sanders and the 2016 presidential election against Donald Trump. The data set for Hillary vs. Bernie is the same data set we used for our previous in class election analysis. The second data set comes from kaggle.com and has the presidential election information by county. I use these data sets to calculate how many percentage points Hillary lost to Bernie and how many she lost to Trump in each state and then take the average to see if there are any interesting trends between the two plots. One such trend I expected to see was that an embarassing loss to Bernie would make it more likely for her to lose the state big to Trump, but for the most part this doesn't seem to be the case.

In [102]:
%matplotlib inline
import pandas as pd
import seaborn as sbn
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as graph_objs
sbn.set()
from bokeh.palettes import RdBu
import sqlalchemy

%load_ext sql

engine = sqlalchemy.create_engine('postgresql://millbr02:@localhost/election')

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [103]:
import plotly.tools as tls
tls.set_credentials_file(username='imoewi01', api_key='b8qvv6pv20')

Importing the data set used in our original election data analysis project.

In [104]:
cr = pd.read_sql('county_results',engine)

In [105]:
#make pivot table to compare Bernie and Hillary
dems = cr[(cr.candidate == 'Bernie Sanders') | (cr.candidate == 'Hillary Clinton')]

In [106]:
compbh = pd.pivot_table(dems, values='votes', index='state', columns = 'candidate', aggfunc=sum)

In [107]:
newest = compbh.reset_index()

In [108]:
Minn = {'state': 'Minnesota', 'Bernie Sanders':118135, 'Hillary Clinton':73510}
newest = newest.append(Minn, ignore_index=True)

Calculating the percentages that Hillary and Bernie had in each state.

In [109]:
newest['HillPerc'] = (newest['Hillary Clinton'])/(newest['Bernie Sanders'] + newest['Hillary Clinton'])
newest['BernPerc'] = (newest['Bernie Sanders'])/(newest['Bernie Sanders'] + newest['Hillary Clinton'])

Calculating the number of percentage points Hillary lost by.

In [110]:
newest = newest.fillna(0)
newest['HillDiff'] = -100*(newest['HillPerc']-newest['BernPerc'])

In [111]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}


Adding a state abbreviation column to the dataframe so we can use plotly's state choropleth. 

In [112]:
ap_li = []
for state in newest['state']:
    for key in states.keys():
        if state==states[key]:
            ap_li.append(key)

In [113]:
newest['StateAbb'] = ap_li

In [114]:
scl = [[-1, 'rgb(0,230,0)'],[-0.6, 'rgb(0,250,150)'],[-0.2, 'rgb(0,250,150)'],\
            [0.2, 'rgb(0,250,250)'],[0.6, 'rgb(0,150,250)'],[1.0, 'rgb(250,0,0)']]

In [115]:
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = newest['StateAbb'],
        z = newest['HillDiff'].astype(float),
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "# Points Bernie Won By"
        )
    ) ]

In [116]:
layout = dict(
        title = 'Hillary vs Bernie by State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )
fig = dict( data=data, layout=layout )

Creating the plot for Hillary vs. Bernie.

In [46]:
url = py.plot( fig, filename='HillPlot')

Importing data from kaggle data set.

In [117]:
final_res = pd.read_csv("votes.csv")

In [118]:
final_res = final_res[['county_name','state_abbreviation','Hillary','Trump']]

In [119]:
final_res = final_res.groupby('state_abbreviation').sum()
final_res = final_res.reset_index()


In [120]:
Alaska = {'state_abbreviation': 'AK', 'Trump':163387, 'Hillary':116454}
final_res = final_res.append(Alaska,ignore_index=True)

Calculating the number of points Trump won by.

In [121]:
final_res['TrumpDiff'] = (-final_res['Hillary']/(final_res['Hillary']+final_res['Trump'])+final_res['Trump']/(final_res['Hillary']+final_res['Trump']))*100

In [122]:
scl2 = [[0, 'rgb(0,0,250)'],[0.375, 'rgb(150,150,150)'],[1, 'rgb(250,0,0)']]

In [123]:
data2 = [ dict(
        type='choropleth',
        colorscale = scl2,
        autocolorscale = False,
        locations = final_res['state_abbreviation'],
        z = final_res['TrumpDiff'].astype(float),
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "# Points Trump Won By"
        )
    ) ]

In [124]:
layout2 = dict(
        title = 'Hillary vs Trump by State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )
fig2 = dict( data=data2, layout=layout2 )

Creating plot for Hillary vs. Trump.

In [99]:
url2 = py.plot( fig2, filename='HillvTrump')

Calculating the average number of points Hillary lost by in each state.

In [125]:
final_res = final_res[['state_abbreviation','TrumpDiff']]
newest = newest[['StateAbb','HillDiff']]
final_res['HillDiff'] = final_res['TrumpDiff']
del final_res['TrumpDiff']
newest['state_abbreviation'] = newest['StateAbb']
del newest['StateAbb']

In [126]:
final = final_res.append(newest)

In [127]:
final = final.groupby('state_abbreviation').mean()
final = final.reset_index()

In [128]:
scl3 = [[0, 'rgb(0,0,250)'],[0.3, 'rgb(150,150,150)'],[1.0, 'rgb(250,0,0)']]

In [129]:
data3 = [ dict(
        type='choropleth',
        colorscale = scl3,
        autocolorscale = False,
        locations = final['state_abbreviation'],
        z = final['HillDiff'].astype(float),
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Average # Points Hillary Lost By"
        )
    ) ]

In [130]:
layout3 = dict(
        title = 'Hillary Results by State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )
fig3 = dict( data=data3, layout=layout3 )

In [131]:
url3 = py.plot( fig3, filename='HillaryByState')

# Hillary vs. Bernie
I found this plot to be rather surprising, since the vicories for each of the candidates seemed much more localized than I realized. Hillary dominated the South and won key swing states like Iowa in the Midwest. Bernie won the majority of the states to the west of the Missouri River, along with his home state of Vermont.

In [132]:
tls.embed(url)

# Hillary vs. Trump by State
This plot shows why Donald Trump will be the next president of the United States. Apart from victories in predominately democratic states like Washington and California, Hillary did not have a lot of major victories. Apart from the west coast and the north east, Trump won almost every state. 

In [133]:
tls.embed(url2)

# Average Points Hillary Lost By
This plot shows the average number of points Hillary lost by in each state. The most significant state for losses for Hillary was Idaho, followed closely by Utah and North Dakota. Hillary lost by around 60 points in both Idaho and Utah against Bernie and lost by around 40 against Trump. Hillary's biggest state for victories was Maryland, with a 26 point win against Trump and a 31 point win against Bernie. This was followed closely by Mississippi, which I found surprising since I expected Mississippi to be rather conservative being in the south. While Trump did have a 19 point win against Hillary there, Hillary beat Bernie by 67 points there. This huge win raises the average so much that it is actually the second highest for average number of points she won by.

In [134]:
tls.embed(url3)